In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab

In [2]:
data = pd.read_csv('PS_2020.11.24_08.25.04.csv', delimiter=',')
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop('index', axis=1, inplace=True)
data.head()

FileNotFoundError: [Errno 2] File PS_2020.11.24_08.25.04.csv does not exist: 'PS_2020.11.24_08.25.04.csv'

In [ ]:
semi_major = data['pl_orbsmax']  #AU
period = data['pl_orbper'] #days 
radius = data['pl_rade'] #earth radii
mass = data['pl_masse']  #earth mass 
print(len(radius))

In [ ]:
data2 = pd.DataFrame({'log(mass)':np.log10(mass), 'log(radius)':np.log10(radius)})
data3 = pd.DataFrame({'log(mass)':np.log10(mass), 'log(radius)':np.log10(radius), 'method':data['discoverymethod']})
data4 = pd.DataFrame({'log(period)':np.log10(period), 'log(semi_major)':np.log10(semi_major), 'method':data['discoverymethod']})

In [ ]:
#plt.figure(figsize=(14,8))
sns.scatterplot(x=data4['log(semi_major)'], y=data4['log(period)'], hue=data3['method'], marker='o')
#selection effect that no planets with a smei major axis above 1 AU is detected - obviously incorrect 

In [ ]:
#plt.figure(figsize=(14,8))
sns.scatterplot(x=data2['log(mass)'], y=data2['log(radius)'], hue=data3['method'], marker='o')

In [ ]:
sns.lmplot(x='log(mass)', y='log(radius)', data=data2, )

In [ ]:
from CompSample import linfit
from scipy.optimize import curve_fit

# Let's try a linear fit to the data (my linfit function definition has been imported from CompSample.py)
popt,pcov = curve_fit(linfit,data2['log(mass)'],data2['log(radius)'],p0=[1])

a = popt[0]
aerr = np.sqrt(pcov[0][0])

print('\033[1m' + "Linear fit has best-fit parameter:\n\tM/L = %.3f +/- %.3f\n" % (a,aerr) + '\033[0m')

In [ ]:
sns.scatterplot(x=data2['log(mass)'], y=data2['log(radius)'], hue=data3['method'], marker='o')
pylab.plot(data2['log(radius)'], linfit(data2['log(radius)'],a),c='k',zorder=2,label='Model')
pylab.legend(loc=4)

In [ ]:
# Import package necessary to determine distribution of data points
# and to be able to colour-code by number density
from CompSample import conflevels, density_scatter
import seaborn as sns
import pandas as pd

# Let's fine regions in M-L parameter space containig 68%, 90% and 95% of the sample
# Note the nbins argument. This sets the number of bins in the M-L parameter space used
# to determine the location of the contours. If we only have 20 data points, nbins=10
# would be too large, since it would only give us 2 data points. Similarly, if we have
# 1000 data points, nbins=10 may be too small
levels = conflevels(data2['log(mass)'],data2['log(radius)'],nbins=80,confints=[0.99,0.7,0.3])

# Let's set the colours of each data point according to the number density at M-L
# Similar considerations apply here in the choice of values used in the bins argument
# This time the bins can have different dx and dy lengths
dencol=density_scatter(data2['log(mass)'], data2['log(radius)'], bins = [30,30])

# Need to store the Stellar mass and lumninosty values as a 'pandas DataFrame'
#MLdata = {'Luminosity':Lum,'Stellar Mass':Mass}
#df = pd.DataFrame(MLdata,columns=['Luminosity','Stellar Mass'])

# Plot the data points, colour coded according to number density
pylab.figure(figsize=(14,6))
pylab.scatter(data2['log(mass)'], data2['log(radius)'], c=dencol, s=10) # c=dencal argument sets colour-coding according to number density
pylab.xlabel('log(mass)')
pylab.ylabel('log(radius)')
# This command plots the contours containing 68%, 90% and 95% or the sample
sns.kdeplot(data=data2,n_levels=levels)